# S&P 500 exposure: VOO on Reg-T margin vs Micro E-mini futures (MES)

This example compares two ways to express S&P 500 exposure:
1) VOO ETF on Reg-T margin (principal-exchange spot)
2) Micro E-mini S&P 500 futures (MES) on variation margin

It runs the same random trade schedule under two leverage factors (`1.0x` and
`2.0x`) so we can see how leverage changes costs and outcomes for each vehicle.

Data window: 2020-01 to 2024-12.

Modeling assumptions:
- VOO uses OHLCV plus dividend/split files and is transformed to a simple
  total-return-like series (dividends embedded into price path).
- MES marks use the daily front contract (highest volume) from the MES contract
  panel, with explicit rolls from `data/MES_roll_dates.csv`.
- Commissions and financing are generated by `IBKRProFixedBroker`.
- USD financing rates use `data/IBKR_USD_benchmark.csv` as a
  compact placeholder schedule for this tutorial.

In [1]:
using Fastback
using Dates
using CSV
using DataFrames
using Random

---------------------------------------------------------

In [2]:
example_dir = normpath(joinpath(@__DIR__, "..", "..", "..", "src", "examples", "5_VOO_vs_MES_comparison"))
isdir(example_dir) || (example_dir = @__DIR__)
data_dir = joinpath(example_dir, "data")

include(joinpath(example_dir, "data.jl"))

# load daily data and align to shared calendar
const BACKTEST_START = Date(2020, 1, 1)
const BACKTEST_END = Date(2024, 12, 31)
voo_df = load_voo_df(
    data_dir;
    start_dt=BACKTEST_START,
    end_dt=BACKTEST_END,
    half_spread=0.01,
)
mes_front_df = load_mes_front_data_df(
    data_dir;
    start_dt=BACKTEST_START,
    end_dt=BACKTEST_END,
    half_spread=0.125,
)
voo_df, mes_front_df = align_on_common_dates(voo_df, mes_front_df)

# MES rolling and fee setup from explicit roll schedule
mes_contract_specs = load_mes_contract_specs(
    data_dir;
    start_dt=first(mes_front_df.dt),
    end_dt=last(mes_front_df.dt),
)
mes_per_contract_fees = Dict(
    spec.symbol => 1.22  # 0.85 + 0.35 + 0.02
    for spec in mes_contract_specs
)

# load USD benchmark schedule
const IBKR_USD_BENCHMARK = load_usd_benchmark_schedule(data_dir);

---------------------------------------------------------

In [3]:
function make_ibkr_profile(usd_benchmark_schedule; credit_no_interest_balance, futures_per_contract)
    IBKRProFixedBroker(
        time_type=Date,
        equity_per_share=0.005,
        equity_min=1.00,
        equity_max_pct=0.01,
        futures_per_contract=futures_per_contract,
        benchmark_by_cash=Dict(:USD => usd_benchmark_schedule),
        borrow_spread=0.015,
        lend_spread=0.005,
        credit_no_interest_balance=credit_no_interest_balance,
    )
end

voo_broker = make_ibkr_profile(
    IBKR_USD_BENCHMARK;
    credit_no_interest_balance=10_000,
    futures_per_contract=Dict{Symbol,Price}(),
)

mes_broker = make_ibkr_profile(
    IBKR_USD_BENCHMARK;
    credit_no_interest_balance=10_000,
    futures_per_contract=mes_per_contract_fees,
);

---------------------------------------------------------

In [4]:
# shared backtest loop
function run_backtest!(
    acc,
    inst,
    df,
    trade_lookup;
    target_notional,
)
    for (i, row) in enumerate(eachrow(df))
        dt = row.dt
        bid, ask = row.bid, row.ask
        last = row.last

        marks = [MarkUpdate(inst.index, bid, ask, last)]
        process_step!(acc, dt; marks=marks, liquidate=true)

        event_no = get(trade_lookup, i, 0)
        if event_no != 0
            target_qty = isodd(event_no) ? calc_base_qty_for_notional(inst, last, target_notional) : 0.0
            pos = get_position(acc, inst)
            delta_qty = target_qty - pos.quantity

            if delta_qty != 0.0
                fill_px = delta_qty > 0 ? ask : bid
                order = Order(oid!(acc), inst, dt, fill_px, delta_qty)
                fill_order!(acc, order, dt=dt, fill_price=fill_px, bid=bid, ask=ask, last=last)
            end
        end
    end

    # close any remaining position at the end
    pos = get_position(acc, inst)
    if has_exposure(pos)
        row = df[end, :]
        fill_px = pos.quantity > 0 ? row.bid : row.ask
        order = Order(oid!(acc), inst, row.dt, fill_px, -pos.quantity)
        fill_order!(acc, order, dt=row.dt, fill_price=fill_px, bid=row.bid, ask=row.ask, last=row.last)
    end

    acc
end

# MES futures backtest with explicit rolls
function run_mes_chain_backtest!(
    acc,
    mes_chain,
    mes_roll_dates,
    df,
    trade_lookup;
    target_notional,
)
    active_idx = 1

    for (i, row) in enumerate(eachrow(df))
        dt = row.dt
        bid, ask = row.bid, row.ask
        last = row.last

        active_inst = mes_chain[active_idx]
        process_step!(acc, dt; marks=[MarkUpdate(active_inst.index, bid, ask, last)], liquidate=true)

        while active_idx < length(mes_chain)
            roll_dt = mes_roll_dates[active_idx]
            dt < roll_dt && break

            next_inst = mes_chain[active_idx+1]
            pos = get_position(acc, active_inst)
            if has_exposure(pos)
                close_px = pos.quantity > 0 ? bid : ask
                open_px = pos.quantity > 0 ? ask : bid
                roll_position!(
                    acc,
                    active_inst,
                    next_inst,
                    dt;
                    close_fill_price=close_px,
                    close_bid=bid,
                    close_ask=ask,
                    close_last=last,
                    open_fill_price=open_px,
                    open_bid=bid,
                    open_ask=ask,
                    open_last=last,
                )
            end

            active_idx += 1
            active_inst = mes_chain[active_idx]
        end

        event_no = get(trade_lookup, i, 0)
        if event_no != 0
            target_qty = isodd(event_no) ? calc_base_qty_for_notional(active_inst, last, target_notional) : 0.0
            pos = get_position(acc, active_inst)
            delta_qty = target_qty - pos.quantity

            if delta_qty != 0.0
                fill_price = delta_qty > 0 ? ask : bid
                order = Order(oid!(acc), active_inst, dt, fill_price, delta_qty)
                fill_order!(acc, order; dt=dt, fill_price=fill_price, bid=bid, ask=ask, last=last)
            end
        end
    end

    # close any remaining positions at the end
    for inst in mes_chain
        pos = get_position(acc, inst)
        if has_exposure(pos)
            row = df[end, :]
            fill_price = pos.quantity > 0 ? row.bid : row.ask
            order = Order(oid!(acc), inst, row.dt, fill_price, -pos.quantity)
            fill_order!(acc, order;
                dt=row.dt,
                fill_price=fill_price,
                bid=row.bid,
                ask=row.ask,
                last=row.last,
                allow_inactive=true,
            )
        end
    end

    acc
end

# helper: deterministic random trade schedule (~20 toggles in/out)
function make_trade_lookup(n_rows; n_events=20, seed=2020)
    rng = MersenneTwister(seed)
    pool = collect(15:(n_rows-15))
    trade_indices = sort(pool[randperm(rng, length(pool))[1:n_events]])
    Dict(idx => i for (i, idx) in enumerate(trade_indices))
end

# account + instrument builders

function build_voo_account(initial_cash, broker)
    acc = Account(;
        funding=AccountFunding.Margined,
        base_currency=CashSpec(:USD),
        time_type=Date,
        broker=broker,
    )
    deposit!(acc, :USD, initial_cash)

    voo = register_instrument!(
        acc,
        spot_instrument(
            :VOO,
            :VOO,
            :USD;
            time_type=Date,
            base_tick=1.0,
            base_digits=0,
            quote_tick=0.01,
            quote_digits=2,
            margin_requirement=MarginRequirement.PercentNotional,
            margin_init_long=0.50,
            margin_init_short=0.35,  # equals 135% collateral-style notion
            margin_maint_long=0.25,
            margin_maint_short=0.20,  # equals 120% collateral-style notion
        ),
    )

    acc, voo
end

function build_mes_account(initial_cash, broker, mes_specs)
    acc = Account(
        funding=AccountFunding.Margined,
        base_currency=CashSpec(:USD),
        time_type=Date,
        broker=broker,
    )
    deposit!(acc, :USD, initial_cash)

    mes_chain = Instrument{Date}[]
    mes_roll_dates = Date[]
    for spec in mes_specs
        inst = register_instrument!(
            acc,
            future_instrument(
                spec.symbol,
                :MES,
                :USD;
                time_type=Date,
                base_tick=1.0,
                base_digits=0,
                quote_tick=0.25,
                quote_digits=2,
                multiplier=5.0,
                margin_requirement=MarginRequirement.FixedPerContract,
                margin_init_long=2_800.0,
                margin_init_short=2_800.0,
                margin_maint_long=2_421.0,
                margin_maint_short=2_421.0,
                expiry=spec.expiry,
            ),
        )
        push!(mes_chain, inst)
        push!(mes_roll_dates, spec.roll_date)
    end

    acc, mes_chain, mes_roll_dates
end

# summarize results (costs + net equity)

function summarize(acc, label, initial_cash, leverage_factor)
    end_equity = equity(acc, cash_asset(acc, :USD))
    pnl = end_equity - initial_cash
    commissions = sum(t.commission_settle for t in acc.trades, init = 0.0)
    roll_trades = count(t -> t.reason == TradeReason.Roll, acc.trades)
    lend_interest = sum(cf.amount for cf in acc.cashflows if cf.kind == CashflowKind.LendInterest, init=0.0)
    borrow_interest = -sum(cf.amount for cf in acc.cashflows if cf.kind == CashflowKind.BorrowInterest, init=0.0)
    net_interest = lend_interest - borrow_interest
    borrow_fees = sum(cf.amount for cf in acc.cashflows if cf.kind == CashflowKind.BorrowFee, init=0.0)

    (
        leverage=leverage_factor,
        instrument=label,
        target_notional=round(leverage_factor * initial_cash, digits=2),
        trades=length(acc.trades),
        roll_trades=roll_trades,
        end_equity=round(end_equity, digits=2),
        pnl=round(pnl, digits=2),
        commissions=round(commissions, digits=2),
        lend_interest=round(lend_interest, digits=2),
        borrow_interest=round(borrow_interest, digits=2),
        net_interest=round(net_interest, digits=2),
        borrow_fees=round(borrow_fees, digits=2),
    )
end

summarize (generic function with 1 method)

---------------------------------------------------------

In [5]:
# run scenarios

initial_cash = 200_000.0
leverage_factors = (1.0, 2.0)
trade_lookup = make_trade_lookup(nrow(voo_df); n_events=20, seed=2020)

rows = []
for leverage_factor in leverage_factors
    target_notional = leverage_factor * initial_cash

    acc_voo, voo = build_voo_account(initial_cash, voo_broker)
    run_backtest!(acc_voo, voo, voo_df, trade_lookup; target_notional=target_notional)
    push!(rows, summarize(acc_voo, "VOO (Reg-T margin)", initial_cash, leverage_factor))

    acc_es, mes_chain, mes_roll_dates = build_mes_account(initial_cash, mes_broker, mes_contract_specs)
    run_mes_chain_backtest!(
        acc_es,
        mes_chain,
        mes_roll_dates,
        mes_front_df,
        trade_lookup;
        target_notional=target_notional,
    )
    push!(rows, summarize(acc_es, "MES (futures margin)", initial_cash, leverage_factor))
end

summary = DataFrame(rows)
sort!(summary, [:leverage, :instrument])
summary

Row,leverage,instrument,target_notional,trades,roll_trades,end_equity,pnl,commissions,lend_interest,borrow_interest,net_interest,borrow_fees
,Float64,String,Float64,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1.0,MES (futures margin),200000.0,28,8,2.6681e5,66809.9,322.08,25853.2,-0.0,25853.2,0.0
2,1.0,VOO (Reg-T margin),200000.0,20,0,2.70104e5,70104.1,51.93,24315.9,-0.0,24315.9,0.0
3,2.0,MES (futures margin),400000.0,28,8,3.15346e5,1.15346e5,671.0,30593.3,-0.0,30593.3,0.0
4,2.0,VOO (Reg-T margin),400000.0,20,0,3.1622e5,1.1622e5,103.9,28064.9,3415.45,24649.4,0.0


---------------------------------------------------------

In [6]:
# compact view: leverage effect within each instrument
leverage_effect_wide = combine(groupby(summary, :instrument)) do sdf
    s1 = sdf[sdf.leverage.==1.0, :]
    s2 = sdf[sdf.leverage.==2.0, :]
    @assert nrow(s1) == 1 && nrow(s2) == 1

    (
        pnl_1x=s1.pnl[1],
        pnl_2x=s2.pnl[1],
        pnl_delta_2x_minus_1x=round(s2.pnl[1] - s1.pnl[1], digits=2),
        roll_trades_1x=s1.roll_trades[1],
        roll_trades_2x=s2.roll_trades[1],
        comm_1x=s1.commissions[1],
        comm_2x=s2.commissions[1],
        lend_interest_1x=s1.lend_interest[1],
        lend_interest_2x=s2.lend_interest[1],
        borrow_interest_1x=s1.borrow_interest[1],
        borrow_interest_2x=s2.borrow_interest[1],
        net_interest_1x=s1.net_interest[1],
        net_interest_2x=s2.net_interest[1],
    )
end

metric_cols = names(leverage_effect_wide, Not(:instrument))
leverage_effect = DataFrame(metric=String.(metric_cols))
for row in eachrow(leverage_effect_wide)
    leverage_effect[!, Symbol(row.instrument)] = [row[col] for col in metric_cols]
end

leverage_effect

Row,metric,MES (futures margin),VOO (Reg-T margin)
,String,Real,Real
1,pnl_1x,66809.9,70104.1
2,pnl_2x,1.15346e5,1.1622e5
3,pnl_delta_2x_minus_1x,48536.2,46116.3
4,roll_trades_1x,8,0
5,roll_trades_2x,8,0
6,comm_1x,322.08,51.93
7,comm_2x,671.0,103.9
8,lend_interest_1x,25853.2,24315.9
9,lend_interest_2x,30593.3,28064.9
